In [ ]:
from urllib.request import urlopen
from html.parser import HTMLParser
import os
import wget

In [ ]:
class MyHTMLParser(HTMLParser):
    def __init__(self, predicate):
        super().__init__()
        self.journals = []
        self.state = False
        self.predicate = predicate
        
    def handle_starttag(self, tag, attrs):
        self.state = tag == 'a'
        
    def handle_data(self, data):
        text = data.strip()
        if self.state and len(text) > 0 and self.predicate(text):
            self.journals.append(data)

class PDFHTMLParser(HTMLParser):
    def __init__(self, predicate):
        super().__init__()
        self.journals = []
        self.predicate = predicate
        
    def handle_data(self, data):
        text = data.strip()
        if self.predicate(text):
            self.journals.append(data)

# Выгружение всех журналов

In [ ]:
url_root = "https://socionet.ru/~cyrcitec/j-neicon/"
html_root = urlopen(url_root).read().decode()

journals_catalogue = MyHTMLParser(lambda x: x.endswith('/'))
journals_catalogue.feed(html_root)

In [ ]:
len(journals_catalogue.journals)

# Из каждого журнала выбираем статью (например, первую) и ищем для нее PDF

In [ ]:
try:
    os.mkdir('originals')
except:
    pass

In [ ]:
for journal in journals_catalogue.journals:
    html_journal = urlopen(''.join([url_root, journal])).read().decode()
    journal_catalogue = MyHTMLParser(lambda x: x.endswith('.json'))
    journal_catalogue.feed(html_journal)
    
    try:
        os.mkdir(f'originals/{journal[:-1]}')
    except:
        pass
    
    counter = 0
    for paper_name in journal_catalogue.journals:
        counter += 1
        paper_txt = '/'.join(['http://no-xml.socionet.ru/~cyrcitec/jfmt.cgi?file=j-neicon', 
                              journal[:-1], paper_name])
        print(paper_txt)
        wget.download(paper_txt, out=f'originals/{journal[:-1]}/{paper_name[:-5]}.txt')
        
        paper_code = ':'.join(["spz", "neicon", journal[:-1], paper_name[:-5]])
        
        url_pdf_page = ''.join(["https://socionet.ru/publication.xml?h=", paper_code])
        
        html_pdf = urlopen(url_pdf_page).read().decode()
        pdf_links = PDFHTMLParser(lambda x: x.endswith('.pdf'))
        pdf_links.feed(html_pdf)
        
        try:
            wget.download(pdf_links.journals[-1], out=f'originals/{journal[:-1]}/{paper_name[:-5]}.pdf')
        except:
            pass
        
        if counter > 0:
            break